In [1]:
import numpy as np
import gym
import tensorflow as tf
tf.reset_default_graph()

c:\tools\anaconda3\envs\ztdl-tf-1\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\tools\anaconda3\envs\ztdl-tf-1\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\tools\anaconda3\envs\ztdl-tf-1\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\tools\anaconda3\envs\ztdl-tf-1\lib\site-packages\tensorflow\python\framework

In [2]:
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

In [3]:
env = gym.make("FrozenLake-v0")

In [4]:
input_size = env.observation_space.n
output_size = env.action_space.n
hidden_size = 16

In [5]:
def one_hot(x):
    return np.eye(input_size)[x:x+1]

In [6]:
class Q:
    def __init__(self):
        self.env = env
        self._build_net()
        
    def _build_net(self):
        self.inputs = tf.placeholder(tf.float32, [None, input_size], "inputs")
        self.targets = tf.placeholder(tf.float32, [None, output_size], "targets")
        
        weights_1 = tf.get_variable("weights_1", [input_size, hidden_size])
        biases_1 = tf.get_variable("biases_1", [hidden_size])
        outputs_1 = tf.nn.relu(tf.matmul(self.inputs, weights_1) + biases_1)
        
        weights_2 = tf.get_variable("weights_2", [hidden_size, hidden_size])
        biases_2 = tf.get_variable("biases_2", [hidden_size])
        outputs_2 = tf.nn.relu(tf.matmul(outputs_1, weights_2) + biases_2)

        weights_3 = tf.get_variable("weights_3", [hidden_size, hidden_size])
        biases_3 = tf.get_variable("biases_3", [hidden_size])
        outputs_3 = tf.nn.relu(tf.matmul(outputs_2, weights_3) + biases_3)

        weights_4 = tf.get_variable("weights_4", [hidden_size, output_size])
        biases_4 = tf.get_variable("biases_4", [output_size])
        
        self.outputs = tf.matmul(outputs_3, weights_4) + biases_4
        
        self.mean_loss = tf.losses.mean_squared_error(labels=self.targets, predictions=self.outputs) / 2.
        
        self.optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.mean_loss)
        
    def predict(self, state):
        out = sess.run(self.outputs, feed_dict={self.inputs: state.reshape(1, input_size)})
        return out
    
    def train(self, state, action_target):
        sess.run([self.optimize],
                 feed_dict={self.inputs: state.reshape(-1, input_size), self.targets: action_target.reshape(-1, output_size)})

In [7]:
q = Q()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [8]:
NUM_EPISODES = 20000
EPSILON = 0.3
DIS = 0.9

In [9]:
sess = tf.Session()
init = tf.global_variables_initializer()
# init = tf.initialize_all_variables()
sess.run(init)

In [10]:
for episode in range(NUM_EPISODES):
    s = env.reset()
    
    if episode % 100 == 0:
        print(episode)
    
    while True:
        actions = q.predict(one_hot(s))
        
        e = np.random.rand()
        if e < EPSILON:
            a = env.action_space.sample()
        else:
            a = np.argmax(actions)

        s1, reward, done, _ = env.step(a)

        if done:
            actions[0, a] = reward
        else:
            next_actions = q.predict(one_hot(s1))
            actions[0, a] = reward + DIS * np.max(next_actions)
            
        q.train(one_hot(s), actions)
        
        if done:
            break
            
        s = s1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [11]:
def check_performance():
    total_reward_sum = 0.0
    
    for episode in range(NUM_EPISODES):
        s = env.reset()
        reward_sum = 0
    
        while True:
            actions = q.predict(one_hot(s))
            a = np.argmax(actions)
            s1, reward, done, _ = env.step(a)
            reward_sum += reward
            if done:
                break
            s = s1
        
        total_reward_sum += reward_sum
    
    print("reward_sum avg = ", total_reward_sum / NUM_EPISODES)

In [12]:
check_performance()

reward_sum avg =  0.0926
